# <center>Using Graph Theory in Nuclear Physics</center>

<center>by Chi-En Teh (Fanurs)</center>

In this report, I would like to briefly highlight two applications of graph theory, or interchangeably, network theory, in nuclear physics. The first application is one that I had encountered in my proposed project but was not aware of it until the introduction of graph theory in class. The second application is one that could be useful when dealing with a long nested chains of nuclear reactions.

## Section 1: Clustering of nucleons from quantum molecular dynamics (QMD)

The title of my proposed project is "Constraining Skyrme with Bayesian inference on heavy-ion collisions and Gaussian-emulated ImQMD", where ImQMD stands for "Improved Quantum Molecular Dynamics". Both QMD and ImQMD are simulation techniques used for modeling the dynamics of particles. In the case of nuclear physics, QMD can be applied by viewing an atomic nucleus as a composite system of neutrons and protons.

<img src="https://cerncourier.com/wp-content/uploads/2013/04/CCfir5_04_13.jpg" width="70%">

Image from: [CERN Courier](https://cerncourier.com/wp-content/uploads/2013/04/CCfir5_04_13.jpg)

To predict the fragments from collisions between heavy nuclei (heavy-ion collision), a computer program on QMD would typically be computing and tracking the motions of individual nucleons, a collective term for neutrons and protons. However, we do not expect the fragments to consist of only individual nucleons, i.e. for a collision between nucleus $A$ of $40$ nucleons and nucleus $B$ of $64$ nucleons, we do not observe $40 + 64 = 104$ of individual fragments being emitted. A reasonable estimate would be around $40$ to $70$ pieces of fragments due to quite a lot of "stickiness" among nearby nucleons. There is an analogy that I like using when explaining to the public: Heavy-ion collision is pretty much like smashing two rolls of sushi rice in mid-air with high speed. We do expect them breaking into pieces, but as long as the sushi rolls are freshly made, there will always be some rices that stick together, forming the so-called "clusters".

<img src="https://cdn.statically.io/img/nextshark.com/wp-content/uploads/2016/01/31.png?quality=100" width="40%">

Image from: [NEXTSHARK](https://cerncourier.com/wp-content/uploads/2013/04/CCfir5_04_13.jpg)

In the ImQMD code that I am using for this project, the clustering process is done after all the tedious QMD calculations. This is the stage where positions and momentums of all individual nucleons have already been established. It remains for us to group the nucleons in a way that matches the experiment. Unsurprisingly, but a little surprising to me at first, this can actually be done using graph theory. Specifically, I only need some simple modification to the "nearest neighbor graph" function we did during one of our pre-class assignments.

### A simplified 2D example

The basic idea of ImQMD clustering is to put any pairs of nucleons that are within some "threshold distance" into the same cluster. This "distance" is generally a six-dimensional metric, with three position components and three momentum components. In this example, I would only use an example with two spatial dimensions, $x$ and $y$.

So instead of looking at only the nearest neighbor, like what was done during the pre-class assignment, we would include all particles within the threshold distance as "nearby neighbors", or simply, "neighbors", and group them into a cluster. The following script demonstrates the idea for some randomly generated points. The colors identify the clusters, and the edges indicate the nearby neighbors. Notice that not every pair of nodes within the same cluster is nearby neighbor. In the language of graph theory or `networkx`, we are cluster particles by the graph's _connected components_.

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import networkx as nw

def get_neighbor_graph(particles, threshold=0.1):
    n_particles = len(particles)
    graph = dict()
    for i in range(n_particles):
        neighbors_of_i = set()
        for j in range(n_particles):
            if i == j: continue # skip itself
            if np.linalg.norm(particles[i] - particles[j]) <= threshold:
                neighbors_of_i.add(j)
        graph[i] = neighbors_of_i
    return graph
    
def get_clusters(graph):
    graph = nw.Graph(graph)
    clusters = [*nw.connected_components(graph)]
    return clusters

# generate random particles within a 2D unit (square) cell
n_particles = 100
np.random.seed(0)
data = pd.DataFrame(np.random.random(size=(n_particles, 2)), columns=['x', 'y'])
neighbor_graph = get_neighbor_graph(data.to_numpy())
clusters = get_clusters(neighbor_graph)

# plot results
fig, ax = plt.subplots(dpi=100, figsize=(4,4))
n_clusters = len(clusters)
colors = plt.cm.jet(np.array(range(n_clusters)) / (n_clusters-1))
for i in neighbor_graph.keys():
    for j in neighbor_graph[i]:
        x = [data.loc[i]['x'], data.loc[j]['x']]
        y = [data.loc[i]['y'], data.loc[j]['y']]
        ax.plot(x, y, color='black', linewidth=0.4)
for clust, color in zip(clusters, colors):
    nodes = np.array([data.loc[i] for i in clust])
    ax.scatter(nodes[:,0], nodes[:,1], s=10, color=color, zorder=4)
ax.set_aspect('equal')

## Section 2: Nuclear reaction networks

Nuclear science is basically the modern practice of alchemy, but in very small quantities. Unlike most chemical reactions at atomic level, nuclear reactions often involve changing elements or isotopes. In fact, any elements heavier than helium or lithium are believed to originate from some nuclear reactions in the burning cores of stars or supernova explosions that took place billions of years ago.

A nucleus is specified by a proton number $Z$ and a neutron number $N$. This suggests the nuclear community to plot all species of nuclei onto a two-dimensional grid, conventionally with the horizontal axis being $N$ and the vertical axis being $Z$. In this way, a nuclear reaction can be represented as a movement from one square to another, just like the checkers or chess. The language of graph theory then becomes a natural way to describe a long chain of reactions. For instance, a tree can be used to represent all possible nuclear reactions from a mother nucleus, with each of the nucleus being a node, and the nuclear reaction from one nucleus to another being a directed edge. Physical quantities associated with the reactions, e.g. the probability of occurrence, can also be assigned to the edges.

<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/4/4e/S-process-elem-Ag-to-Sb.svg/800px-S-process-elem-Ag-to-Sb.svg.png" width="70%">

Image from: [wikipedia](https://upload.wikimedia.org/wikipedia/commons/thumb/4/4e/S-process-elem-Ag-to-Sb.svg/800px-S-process-elem-Ag-to-Sb.svg.png)

One interesting nuclear reaction network that has attracted great attention over the past decades is the "slow neutron-capture process", or simply, the "s-process". The s-process is currently believed to be the mechanism of nucleosynthesis (i.e. creation) of many elements heavier than iron. The above figure shows a possible s-process from silver (Ag) to antimony (Sb). Nonetheless, while many works have mentioned the term "network" in their papers [1-3], I could not find many works that have used network theory extensively except for two: First, we have SkyNet, an open-source modular library that can calculate the nucleosynthesis evolution in various astrophysical scenarios [4]; Second, we have an attempt to map nuclear decays onto a complex network [5]. My guess for the lack of literature is that while network theory has been made powerful, the current nuclear physics community might be limited by our knowledge about individual isotopes.

---
# References

[1] Bisterzo, S., Gallino, R., Straniero, O., Cristallo, S., & Käppeler, F. (2010). s-Process in low-metallicity stars–I. Theoretical predictions. Monthly Notices of the Royal Astronomical Society, 404(3), 1529-1544. doi: [https://doi.org/10.1111/j.1365-2966.2010.16369.x](https://doi.org/10.1111/j.1365-2966.2010.16369.x)<br>
[2] Käppeler, F., Gallino, R., Bisterzo, S., & Aoki, W. (2011). The s process: Nuclear physics, stellar models, and observations. Reviews of Modern Physics, 83(1), 157. doi: [https://doi.org/10.1103/RevModPhys.83.157](https://doi.org/10.1103/RevModPhys.83.157)<br>
[3] Frischknecht, U., Hirschi, R., & Thielemann, F. K. (2012). Non-standard s-process in low metallicity massive rotating stars. Astronomy & astrophysics, 538, L2. doi: [https://doi.org/10.1051/0004-6361/201117794](https://doi.org/10.1051/0004-6361/201117794)<br>
[4] Jonas Lippuner and Luke F. Roberts (2017). SkyNet: A Modular Nuclear Reaction Network Library. The Astrophysical Journal Supplement Series, 233(18). doi: [https://doi.org/10.3847/1538-4365/aa94cb](https://doi.org/10.3847/1538-4365/aa94cb)<br>
[5]  Yong, L., Jin-Qing, F., & Qiang, L. (2012). Mapping Nuclear Decay to a Complex Network. Communications in Theoretical Physics, 57(3), 490. doi: [https://doi.org/10.1088/0253-6102/57/3/25](https://doi.org/10.1088/0253-6102/57/3/25)<br>